Nous allons commencer à créer le dataset comme nous l'avons fait dans le dernier TD.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import glob
from scipy.io import wavfile
from tqdm  import tqdm
import matplotlib.pyplot as plt

dataset_path = "/content/drive/My Drive/IA_MIAGE/google-speech-dataset/"
all_classes = os.listdir(dataset_path)
all_classes.remove('.git')
all_classes.remove('bird') #some outliers in dataset
all_classes.remove('_lab_ressources_')
all_classes.remove('_background_noise_')
chosen_classes = ["sheila", "nine"]

In [ ]:
X = []
for class_name in chosen_classes:
  class_path = dataset_path + class_name
  for audio_path in glob.glob(class_path + "/*.wav"):
    X.append(audio_path)

In [ ]:
y = []
for audio_path in X:
  y.append(audio_path.split('/')[-2])
print(y)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(...)## A completer avec le code du dernier TD
X_test, X_val, y_test, y_val = train_test_split(...)## A completer avec le code du dernier TD

print('training samples:',len(X_train))
print('testing samples:',len(X_test))
print('validation samples:',len(X_val))

In [ ]:
def normalize(audio_signal):
  audio_signal = ## A completer avec le code du dernier TD
  max_value = ## A completer avec le code du dernier TD
  norm_signal = ## A completer avec le code du dernier TD
  return norm_signal

In [ ]:
def add_padding(audio_signal, desired_len):
  
  length_signal = len(audio_signal)
  if length_signal < desired_len:
    zeros = np.zeros(desired_len)
    start_signal =## A completer avec le code du dernier TD
    zeros[start_signal: start_signal + length_signal] = audio_signal
    return zeros
    
  return audio_signal

Maintenant, nous pouvons lire les signaux audio de notre ensemble de données. Nous normalisons tous les audios et ajoutons le padding aux échantillons de moins d'une minute.

In [ ]:
def get_signal(signal_path):
  
  rate, sig = wavfile.read(filename=signal_path)
  
  try:
    sig = sig[:, 0]
  except:
    pass
  
  #normalization
  sig = normalize(sig)

  #standardization of sizes
  sig = add_padding(sig,16000)
  
  return sig

In [ ]:
X_train_waveform = [get_signal(path) for path in tqdm(X_train,position=0)]
X_test_waveform = [get_signal(path) for path in tqdm(X_test,position=0)]

In [ ]:
X_train_waveform = np.asarray(X_train_waveform)
X_test_waveform = np.asarray(X_test_waveform)

scikit fournit également des bibliothèques pour les algorithmes d'apprentissage automatique. 

Vous pouvez d'abord essayer de classer les audios en utilisant directement le signal audio.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_waveform, y_train)

Comme chaque audio a une longueur de 16000 points, knn doit créer un espace de 16000 dimensions. Pour le test, il doit calculer la distance dans toutes les dimensions, c'est pourquoi la partie test peut prendre quelques secondes.

In [ ]:
knn.score(X_test_waveform, y_test)

Essayez différentes valeurs de k, pour trouver la valeur qui donne le meilleur résultat.

In [ ]:
k_to_try = ## A Completer
knn = KNeighborsClassifier(n_neighbors=k_to_try)
knn.fit(X_train_waveform, y_train)
knn.score(X_test_waveform, y_test)

Avec la fonction suivante, vous pouvez essayer différentes valeurs de k et tracer les résultats. Les paramètres de la fonction sont les suivants: 

from_k: Valeur initiale de k que vous voulez essayer

to_k: Valeur finale de k que vous voulez essayer

X_train: dataset d'entrainement

X_test: dataset de test

In [ ]:
def check_results_different_k(from_k, to_k, X_train, X_test):
  scores = []
  k_values = []
  for k in range(from_k, to_k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))
    k_values.append(k)
  plt.plot(k_values, scores)
  plt.show()

In [ ]:
check_results_different_k(3, 10, X_train_waveform, X_test_waveform)

Une stratégie pour réduire la taille du signal consiste à faire de l'échantillonnage.

Faire un échantillonnage signifie prendre une valeur pour chaque échantillon. Par exemple, si vous faites un échantillonnage de 10, vous prenez la première valeur, puis vous sautez 9 valeurs et enfin vous prenez la valeur 11.

In [ ]:
def sampling(X, sampling_step):
  signal_length = X.shape[1]
  new_size = signal_length / sampling_step
  indices_to_take = range(0, signal_length, sampling_step)
  sampled_X = X[:,indices_to_take]
  return sampled_X

In [ ]:
sampling_step = 20
X_train_sampled = sampling(X_train_waveform, sampling_step)
X_test_sampled = sampling(X_test_waveform, sampling_step)

print("Taille d'audio avant echantioolange: ", X_train_waveform.shape[1])
print("Taille d'audio apres echantioolange: ", X_train_sampled.shape[1])

In [ ]:
knn_sampled = KNeighborsClassifier(n_neighbors=3)
knn_sampled.fit(X_train_sampled, y_train)

In [ ]:
knn_sampled.score(X_test_sampled, y_test)

Est-ce plus rapide ?

Les résultats sont-ils meilleurs ?

Essayez différents numéros d'échantillonnage

In [ ]:
sampling_step = ##A Completer
X_train_sampled = sampling(X_train_waveform, sampling_step)
X_test_sampled = sampling(X_test_waveform, sampling_step)
knn_sampled = KNeighborsClassifier(n_neighbors=3)
knn_sampled.fit(X_train_sampled, y_train)
knn_sampled.score(X_test_sampled, y_test)

N'oubliez pas que vous pouvez utiliser la fonction *check_results_different_k* pour vérifier quelle est la meilleure valeur k

In [ ]:
check_results_different_k(..., ..., X_train_sampled, X_test_sampled) # A completer

Voyez-vous les problèmes de l'échantillonnage ? Nous perdons beaucoup d'informations qui pourraient être importantes pour prendre la décision de classification.

Maybe it would be more clever to remove the frequencies where there is no information.

Si vous vous souvenez, un moyen de voir les fréquences du signal est d'utiliser le spectrogramme

*scipy* fournit le *signal* de bibliothèque qui contient de nombreuses fonctions utiles pour traiter les signaux.

Vous pouvez voir le spectrogramme de certains audios.

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt

sample_to_analyze = 100

freqs, times, spectrogram = signal.spectrogram(X_train_waveform[sample_to_analyze])

plt.pcolormesh(times, freqs, spectrogram, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar()
plt.show()

In [ ]:
sample_to_analyze = ## A Completer

freqs, times, spectrogram = signal.spectrogram(X_train_waveform[sample_to_analyze])

plt.pcolormesh(times, freqs, spectrogram, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.colorbar()
plt.show()

Après avoir regardé les spectrogrammes de plusieurs audios de notre ensemble de données, vous pouvez voir qu'il y a certaines fréquences pour lesquelles il n'y a pas d'information. 
 
Vous pouvez définir une limite et vous ne prenez que les valeurs en dessous de ces fréquences.

In [ ]:
limit_freq = # A completer
freqs_to_keep = np.where(freqs <= limit_freq)[0]

C'est la fonction qui permet de filtrer les fréquences, elle ne prendra que les valeurs inférieures à la limite que vous avez définie auparavant.

In [ ]:
def filter_frequencies(X, freqs_to_keep):
  new_X = []
  for signal_wf in X:
    freqs, times, spectrogram = signal.spectrogram(signal_wf)
    filtered_spectrogram  = spectrogram[freqs_to_keep]
    flatten_spectrogram = filtered_spectrogram.flatten()
    new_X.append(flatten_spectrogram)
  return np.asarray(new_X)

In [ ]:
X_train_filtered = filter_frequencies(X_train_waveform, freqs_to_keep)
X_test_filtered = filter_frequencies(X_test_waveform, freqs_to_keep)

In [ ]:
knn_filtered = KNeighborsClassifier(n_neighbors=5)
knn_filtered.fit(X_train_filtered, y_train)

In [ ]:
knn_filtered.score(X_test_filtered, y_test)

Est-ce plus rapide ?

Les résultats sont-ils meilleurs ?

N'oubliez pas que vous pouvez utiliser la fonction *check_results_different_k* pour vérifier quelle est la meilleure valeur k

In [ ]:
check_results_different_k(..., ..., X_train_filtered, X_test_filtered) # A completer

Comme nous l'avons vu dans le cours, les fonctionnalités du MFCC sont très populaires pour la reconnaissance vocale. 

sonopy fournit une fonction permettant d'extraire les caractéristiques du MFCC.

In [ ]:
!pip install sonopy
from sonopy import mfcc_spec

In [ ]:
def get_mfcc_from_signal(signal):

  #extract MFCCs features
  single_mfcc = mfcc_spec(signal, 16000, window_stride=(400, 160), fft_size=512, num_filt=20, num_coeffs=13).T

  return single_mfcc

In [ ]:
X_train_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_train_waveform,position=0)]
X_test_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_test_waveform,position=0)]

In [ ]:
plt.figure(figsize=(12,4))
plt.imshow(X_train_mfcc[0],cmap='Spectral')
plt.title("MFCC (transposed)")
plt.ylabel('MFCC Coffiencts')
plt.xlabel('Time [sec]')
plt.show()

print('shape (transposed):',np.array(X_train_mfcc[0]).shape)

In [ ]:
X_train_mfcc = np.asarray(X_train_mfcc)
X_test_mfcc = np.asarray(X_test_mfcc)

X_train_mfcc_flatten = X_train_mfcc.reshape(X_train_mfcc.shape[0], X_train_mfcc.shape[1] * X_train_mfcc.shape[2])
X_test_mfcc_flatten = X_test_mfcc.reshape(X_test_mfcc.shape[0], X_test_mfcc.shape[1] * X_test_mfcc.shape[2])

In [ ]:
knn_mfcc = KNeighborsClassifier(n_neighbors=5)
knn_mfcc.fit(X_train_mfcc_flatten, y_train)

In [ ]:
knn_mfcc.score(X_test_mfcc_flatten, y_test)

Est-ce plus rapide ?

Les résultats sont-ils meilleurs ?

N'oubliez pas que vous pouvez utiliser la fonction *check_results_different_k* pour vérifier quelle est la meilleure valeur k

In [ ]:
check_results_different_k(..., ..., X_train_mfcc_flatten, X_test_mfcc_flatten) # A Completer

Le MFCC fournit 13 coefficients, peut-être que certains d'entre eux ne sont pas utiles pour classer vos audios.

La fonction suivante prend du coefficient *start_coef* au coefficient *end_coef*.

In [ ]:
def take_mfcc_coefs(start_coef, end_coef, X_mfcc):
  return X_mfcc[:, start_coef: end_coef]


In [ ]:
from_coef = 5
to_coef = 13
X_train_mfcc_some_coef = take_mfcc_coefs(from_coef, to_coef, X_train_mfcc)
X_test_mfcc_some_coef = take_mfcc_coefs(from_coef, to_coef, X_test_mfcc)

X_train_mfcc_some_coef_flatten = X_train_mfcc_some_coef.reshape(X_train_mfcc_some_coef.shape[0], X_train_mfcc_some_coef.shape[1] * X_train_mfcc_some_coef.shape[2])
X_test_mfcc_some_coef_flatten = X_test_mfcc_some_coef.reshape(X_test_mfcc_some_coef.shape[0], X_test_mfcc_some_coef.shape[1] * X_test_mfcc_some_coef.shape[2])


In [ ]:
knn_mfcc = KNeighborsClassifier(n_neighbors=5)
knn_mfcc.fit(X_train_mfcc_some_coef_flatten, y_train)
knn_mfcc.score(X_test_mfcc_some_coef_flatten, y_test)

Les résultats sont-ils meilleurs ?

N'oubliez pas que vous pouvez utiliser la fonction *check_results_different_k* pour vérifier quelle est la meilleure valeur k

In [ ]:
check_results_different_k(..., ..., X_train_mfcc_some_coef_flatten, X_test_mfcc_some_coef_flatten) # A Completer

Vous pouvez essayer de vous débarrasser de certains coefficients pour voir si les résultats s'améliorent

In [ ]:
from_coef = ## A Completer
to_coef = ## A Completer
X_train_mfcc_some_coef = take_mfcc_coefs(from_coef, to_coef, X_train_mfcc)
X_test_mfcc_some_coef = take_mfcc_coefs(from_coef, to_coef, X_test_mfcc)

X_train_mfcc_some_flatten = X_train_mfcc_some_coef.reshape(X_train_mfcc_some_coef.shape[0], X_train_mfcc_some_coef.shape[1] * X_train_mfcc_some_coef.shape[2])
X_test_mfcc_some_coef_flatten = X_test_mfcc_some_coef.reshape(X_test_mfcc_some_coef.shape[0], X_test_mfcc_some_coef.shape[1] * X_test_mfcc_some_coef.shape[2])

knn_mfcc = KNeighborsClassifier(n_neighbors=5)
knn_mfcc.fit(X_train_mfcc_some_flatten, y_train)
knn_mfcc.score(X_test_mfcc_some_coef_flatten, y_test)

sklearn fournit d'autres algorithmes basés sur les voisins come NearestCentroid.

In [ ]:
from sklearn.neighbors import NearestCentroid

nc = NearestCentroid()
nc.fit(X_train_waveform, y_train)
nc.score(X_test_waveform, y_test)

Essayez ***NearestCentroid*** avec toutes les données (X_filtered, X_sampled, X_mfcc) que nous avons créées avant pour ***KNeighbors***

In [ ]:
## A completer

Il est important d'analyser ce qui s'est passé si nous utilisons deux classes très similaires. Un simple algorithme basé sur les voisins suffit-il pour classer les classes difficiles ?

Vous pouvez essayer de classer deux classes similaires comme ***three*** et ***tree***.

In [ ]:
chosen_classes = ["tree", "three"]

In [ ]:
X = []
for class_name in chosen_classes:
  class_path = dataset_path + class_name
  for audio_path in glob.glob(class_path + "/*.wav"):
    X.append(audio_path)

In [ ]:
y = []
for audio_path in X:
  y.append(audio_path.split('/')[-2])
print(y)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size=0.2, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test)

print('training samples:',len(X_train))
print('testing samples:',len(X_test))
print('validation samples:',len(X_val))

In [ ]:
X_train_waveform = [get_signal(path) for path in tqdm(X_train,position=0)]
X_test_waveform = [get_signal(path) for path in tqdm(X_test,position=0)]

In [ ]:
X_train_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_train_waveform,position=0)]
X_test_mfcc = [get_mfcc_from_signal(signal) for signal in tqdm(X_test_waveform,position=0)]

In [ ]:
X_train_mfcc = np.asarray(X_train_mfcc)
X_test_mfcc = np.asarray(X_test_mfcc)

X_train_mfcc_flatten = X_train_mfcc.reshape(X_train_mfcc.shape[0], X_train_mfcc.shape[1] * X_train_mfcc.shape[2])
X_test_mfcc_flatten = X_test_mfcc.reshape(X_test_mfcc.shape[0], X_test_mfcc.shape[1] * X_test_mfcc.shape[2])

In [ ]:
knn_mfcc = KNeighborsClassifier(n_neighbors=5)
knn_mfcc.fit(X_train_mfcc_flatten, y_train)
knn_mfcc.score(X_test_mfcc_flatten, y_test)

Comment sont les résultats ?

Le score est très différent de celui obtenu avec les deux autres classes ?

Est-ce à cause du k ? Peut-être que nous n'utilisons pas le bon k pour ces classes

In [ ]:
check_results_different_k(3, 10, X_train_mfcc_flatten, X_test_mfcc_flatten)